# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
city_csv_file = "../3_output_data/cities.csv"

city_csv_file = pd.read_csv("../3_output_data/cities.csv", header = 0,  dtype = {'City': str, 'Country': str, 'Date': str, 'Lat': float, 'Lng': float, 'Max Temp': float, 'Humidity': float,
       'Cloudiness': float, 'Wind Speed': float})



city_csv_file.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Kupino,RU,1612156093,54.3663,77.2981,3.27,88.0,100.0,6.58
1,Rutana Province,BI,1612156093,-3.8667,30.1000,59.27,98.0,100.0,1.77
2,Saint Anthony,US,1612156093,45.0205,-93.2180,30.20,74.0,90.0,8.05
3,Nyurba,RU,1612156093,63.2842,118.3319,-36.40,81.0,94.0,5.30
4,Saskylakh,RU,1612156093,71.9167,114.0833,-30.91,83.0,64.0,7.58


In [21]:
type(city_data_df)

pandas.core.frame.DataFrame

In [20]:
city_data_df.columns

Index(['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed'],
      dtype='object')

In [5]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
#vacation_criteria_df = pd.DataFrame(city_data_df,  columns = ["City", "Country", "Max Temp", "Humidity", "Cloudiness"]
#new = old[['A', 'C', 'D']].copy().

vacation_criteria_df = city_data_df[['City', 'Country', 'Max Temp', 'Humidity', 'Cloudiness', 'Lat', 'Lat' ]].copy()

vacation_city_df = vacation_criteria_df.loc[(vacation_criteria_df["Cloudiness"]== 0) & (vacation_criteria_df["Max Temp"] >= 70) & (vacation_criteria_df["Max Temp"] <= 80)].dropna()

vacation_city_df

,City,Country,Max Temp,Humidity,Cloudiness,Lat,Lat
35,Bhāyāvadar,IN,75.20,23,0,21.8500,21.8500
39,Patiya,BD,74.28,22,0,22.2931,22.2931
72,Nizwá,OM,73.26,26,0,22.9333,22.9333
74,Edd,ER,77.45,55,0,13.9297,13.9297
82,Nicoya,CR,73.40,68,0,10.1483,10.1483
84,Puerto El Triunfo,SV,71.60,94,0,13.2833,13.2833
110,Tura,IN,73.38,31,0,25.5198,25.5198
138,Lázaro Cárdenas,MX,73.92,83,0,17.9583,17.9583
140,Umluj,SA,72.18,30,0,25.0213,25.0213
154,Myitkyina,MM,78.75,34,0,25.3833,25.3833


In [9]:
vacation_city_df.rename(columns={vacation_city_df.columns[5]: 'Lat'}, inplace = True)

vacation_city_df

,City,Country,Max Temp,Humidity,Cloudiness,Lat,Lat
35,Bhāyāvadar,IN,75.20,23,0,21.8500,21.8500
39,Patiya,BD,74.28,22,0,22.2931,22.2931
72,Nizwá,OM,73.26,26,0,22.9333,22.9333
74,Edd,ER,77.45,55,0,13.9297,13.9297
82,Nicoya,CR,73.40,68,0,10.1483,10.1483
84,Puerto El Triunfo,SV,71.60,94,0,13.2833,13.2833
110,Tura,IN,73.38,31,0,25.5198,25.5198
138,Lázaro Cárdenas,MX,73.92,83,0,17.9583,17.9583
140,Umluj,SA,72.18,30,0,25.0213,25.0213
154,Myitkyina,MM,78.75,34,0,25.3833,25.3833


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
len(vacation_city_df)

22

In [13]:
len(city_data_df )

571

In [32]:
len(vacation_city_sort_df)

22

In [18]:
#df = df[['a', 'y', 'b', 'x']]
vacation_city_df = vacation_city_df[['Country', 'City', 'Max Temp', 'Humidity', 'Cloudiness']]

#rslt_df = details.sort_values(by = ['Name', 'Age'])
vacation_city_sort_df = vacation_city_df.sort_values(by = ['Country', 'City'])

vacation_city_sort_df 

,Country,City,Max Temp,Humidity,Cloudiness
39,BD,Patiya,74.28,22,0
438,BR,Barão de Melgaço,77.00,88,0
323,BR,Boa Vista,78.80,73,0
82,CR,Nicoya,73.40,68,0
74,ER,Edd,77.45,55,0
35,IN,Bhāyāvadar,75.20,23,0
502,IN,Dwārka,74.25,40,0
493,IN,Pāsighāt,73.17,29,0
543,IN,Pūrna,79.65,26,0
372,IN,Tezu,75.07,21,0


In [41]:
city_data_df.columns

Index(['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
gmaps.configure(api_key=g_key)

In [34]:
# Store latitude and longitude in locations
locations = vacation_city_sort_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = airport_df["Airport Rating"].astype(float)

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

KeyError: 'humidity'

In [21]:
city_locations = city_data_df[["Lat", "Lng"]]
# locations
humidity = city_data_df["Humidity"].astype(float)
# humidity
figure_layout = {
    'width' : '1000px',
    'height':'400px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig


SyntaxError: invalid syntax (<ipython-input-21-e1552907f79f>, line 13)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
